In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
url="https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page={}"
target_url=url.format(1)
print(target_url)

https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page=1


In [35]:
r=requests.get(target_url)

In [36]:
#取得したデータをsoupに格納
soup = BeautifulSoup(r.text, 'html.parser')

In [37]:
#物件名の取得
contents=soup.find_all("div", class_="cassetteitem")

In [38]:
len(contents)

30

In [ ]:
#賃貸情報の入手を確認　この段階ではforループを使用しないため、変数contenの中に最初の要素を格納しておく。

In [39]:
content = contents[0]

In [40]:
print(content)

<div class="cassetteitem">
<div class="cassetteitem-detail">
<div class="cassetteitem-detail-object">
<div class="cassetteitem_object">
<div class="cassetteitem_object-item">
<img alt="" class="js-noContextMenu js-linkImage js-scrollLazy js-adjustImg" height="0" rel="https://img01.suumo.com/front/gazo/fr/bukken/716/100355213716/100355213716_gw.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAAAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw%3D%3D" width="0"/>
</div>
</div>
</div>
<div class="cassetteitem-detail-body">
<div class="cassetteitem_content">
<div class="cassetteitem_content-label"><span class="ui-pct ui-pct--util1">賃貸マンション</span></div>
<div class="cassetteitem_content-title">パークホームズ日本橋人形町ザレジデンス</div>
<div class="cassetteitem_content-body">
<ul class="cassetteitem_detail">
<li class="cassetteitem_detail-col1">東京都中央区日本橋人形町２</li>
<li class="cassetteitem_detail-col2">
<div class="cassetteitem_detail-text">東京メトロ日比谷線/人形町駅 歩3分</div>
<div class="cassetteitem_detail-text">都営新宿線/浜町駅 歩5分</div>

In [42]:
# 全ての物件情報を辞書型で保存
d = []

# 全ての物件情報を取得
properties = soup.find_all("div", class_="cassetteitem_content-title")

for property_element in properties:
    property_name = property_element.text.strip()
    
    # 物件の詳細情報を取得
    details = property_element.find_next("ul", class_="cassetteitem_detail")
    address = details.find("li", class_="cassetteitem_detail-col1").text.strip()
    access = [item.text.strip() for item in details.find("li", class_="cassetteitem_detail-col2").find_all("div")]
    age = details.find("li", class_="cassetteitem_detail-col3").text.strip()

    # 物件の各部屋情報を取得
    room_data = []
    table_rows = property_element.find_next("table", class_="cassetteitem_other").find_all("tr")[1:]  # 最初の行はヘッダーなので除外

    for row in table_rows:
        columns = row.find_all("td")
        floor = columns[2].text.strip()
        rent = columns[3].text.strip()
        management_fee = columns[4].text.strip()
        layout = columns[6].text.strip()
        size = columns[7].text.strip()

        room_data.append({
            "階数": floor,
            "家賃": rent,
            "管理費": management_fee,
            "間取り": layout,
            "広さ": size
        })

    # 辞書型で物件情報を保存
    d.append({
        "物件名": property_name,
        "住所": address,
        "アクセス": access,
        "築年数": age,
        "部屋情報": room_data
    })

# 辞書型のデータを表示
for property_info in d:
    print(property_info)


{'物件名': 'パークホームズ日本橋人形町ザレジデンス', '住所': '東京都中央区日本橋人形町２', 'アクセス': ['東京メトロ日比谷線/人形町駅 歩3分', '都営新宿線/浜町駅 歩5分', '東京メトロ半蔵門線/水天宮前駅 歩6分'], '築年数': '築6年\n地下1地上18階建', '部屋情報': [{'階数': '13階', '家賃': '43.5万円', '管理費': '-', '間取り': '3LDK', '広さ': '72.51m2'}]}
{'物件名': '東京メトロ半蔵門線 押上駅 4階建 築2年', '住所': '東京都墨田区向島３', 'アクセス': ['東京メトロ半蔵門線/押上駅 歩7分', '東武伊勢崎線/とうきょうスカイツリー駅 歩3分', '都営浅草線/本所吾妻橋駅 歩10分'], '築年数': '築2年\n4階建', '部屋情報': [{'階数': '4階', '家賃': '11.5万円', '管理費': '4000円', '間取り': '1LDK', '広さ': '30.65m2'}]}
{'物件名': 'オーキッドレジデンス銀座東', '住所': '東京都中央区湊３', 'アクセス': ['東京メトロ有楽町線/新富町駅 歩6分', 'ＪＲ京葉線/八丁堀駅 歩8分', '東京メトロ日比谷線/築地駅 歩9分'], '築年数': '築9年\n6階建', '部屋情報': [{'階数': '5階', '家賃': '19.2万円', '管理費': '15000円', '間取り': '1LDK', '広さ': '44.86m2'}]}
{'物件名': 'アーク銀座京橋', '住所': '東京都中央区八丁堀３', 'アクセス': ['東京メトロ銀座線/京橋駅 歩5分', '都営浅草線/宝町駅 歩2分', 'ＪＲ京葉線/八丁堀駅 歩6分'], '築年数': '築16年\n11階建', '部屋情報': [{'階数': '5階', '家賃': '15万円', '管理費': '8000円', '間取り': '1LDK', '広さ': '44.27m2'}]}
{'物件名': 'ドムス両国', '住所': '東京都墨田区両国３', 'アクセス': ['ＪＲ総武線/両国駅 歩7分', '都営新宿線/森下駅 歩10分', '都営大江戸線/両国駅 歩